<a href="https://colab.research.google.com/github/EdoConti/linkedin_review/blob/main/dataset_cleaning_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install chardet pandas numpy tqdm

In [47]:
import os
import json
import pandas as pd
import chardet as chr
from tqdm import tqdm
from pathlib import Path

In [4]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [23]:
def iter_files(root_dir):
    """
    Recursively yield file paths under root_dir.

    :param root_dir: Path to the root directory to iterate.
    :yield: Full path to each file found.
    """
    for entry in os.listdir(root_dir):
        path = os.path.join(root_dir, entry)
        if os.path.isdir(path):
            # Recurse into subdirectory
            yield from iter_files(path)
        else:
            # It's a file, yield its full path
            yield path

In [55]:
def extract_section(data,idx, section, url_field='general.profileUrl'):
    """
    Pulls out data[rec][section] (which should be a list of dicts)
    and flattens it into a DataFrame, adding a 'url' column.
    """
    items = data[idx].get(section, [])
    if not isinstance(items, list) or not items:
        return pd.DataFrame()
    # json_normalize will flatten nested dicts automatically:
    df = pd.json_normalize(items)
    # get the URL (falling back to 'query' if missing)
    url = data[idx].get('general', {}).get('profileUrl', data[idx].get('query'))
    df['url'] = url or None
    return df

In [8]:
out = 'Bankers_Network/LinkedIn profiling/'
inp = 'AdditionalCollection/'

In [9]:
folders_list=[]
info_general= pd.DataFrame()
jobs = pd.DataFrame()
education = pd.DataFrame()
skills = pd.DataFrame()
interests = pd.DataFrame()

In [13]:
for entry in os.listdir(inp):
    entry_path = os.path.join(inp, entry)
    if os.path.isdir(entry_path) and entry_path.endswith('Done'):
        folders_list.append(entry_path)

In [58]:
for dir in iter_files(inp):
    if not dir.endswith('.json'):
        continue
    split_dir = dir.split('/')
    out_data = 'test_data'+'/'+split_dir[1]+'/extracted/'

    out_dir = Path(out_data)
    out_dir.mkdir(parents=True, exist_ok=True)

    with open(dir, 'rb') as file:
        data = json.load(file)

    print(len(data))

797
800
859
1039
757
891
257
705
1034
1000
993
994
926
1001
999
1107
999
999
1030
998
1001
954
583
996
614


In [49]:
for dir in iter_files(inp):
    if not dir.endswith('.json'):
        continue
    split_dir = dir.split('/')
    out_data = 'test_data'+'/'+split_dir[1]+'/extracted/'

    out_dir = Path(out_data)
    out_dir.mkdir(parents=True, exist_ok=True)

    with open(dir, 'rb') as file:
        data = json.load(file)
    error_records=0

    all_general, all_jobs, all_edu, all_skills, all_interests = [], [], [], [], []

    for idx in tqdm(range(0,len(data))):
        records = data[idx].keys()

        if 'error' in records:
            error_records+=1

        # “general” is a single dict, so wrap it in a list
        if 'general' in records:
            g = data[idx]['general'].copy()
            g['url'] = g.pop('profileUrl')  # rename in one go
            all_general.append(g)

        all_jobs.append(extract_section(data, idx, 'jobs'))
        all_edu.append(extract_section(data, idx, 'schools'))
        all_skills.append(extract_section(data, idx, 'skills'))

        # interests is nested by category
        interests = data[idx].get('interests', {})
        for category, lst in interests.items():
            if lst:
                df_int = pd.json_normalize(lst)
                df_int['category'] = category
                df_int['url'] = data[idx].get('general', {}).get('profileUrl')
                all_interests.append(df_int)

    df_general = pd.DataFrame(all_general) if all_general else pd.DataFrame()
    df_jobs = pd.concat(all_jobs,ignore_index=True) if all_jobs else pd.DataFrame()
    df_education = pd.concat(all_edu,ignore_index=True) if all_edu else pd.DataFrame()
    df_skills = pd.concat(all_skills,ignore_index=True) if all_skills else pd.DataFrame()
    df_interests = pd.concat(all_interests,ignore_index=True) if all_interests else pd.DataFrame()

    df_general.to_csv(out_data+'info_general.csv',encoding='utf-8-sig')
    df_jobs.to_csv(out_data+'jobs.csv',encoding='utf-8-sig')
    df_education.to_csv(out_data+'education.csv',encoding='utf-8-sig')
    df_skills.to_csv(out_data+'skills.csv',encoding='utf-8-sig')
    df_interests.to_csv(out_data+'interests.csv',encoding='utf-8-sig')

100%|██████████| 614/614 [00:03<00:00, 171.78it/s]


In [50]:
x = pd.read_csv('/content/drive/MyDrive/AdditionalCollection/2_Collection_Lukas_Done/extracted/education.csv')

In [51]:
y = pd.read_csv('/content/drive/MyDrive/test_data/2_Collection_Lukas_Done/extracted/education.csv')

In [52]:
print(len(x),len(y))

10110 2049


In [53]:
x.head()

,Unnamed: 0,schoolUrl,schoolName,logoUrl,degree,dateRange,url,description
0,0,https://www.linkedin.com/company/5290/,The Wharton School,https://media.licdn.com/dms/image/C4E0BAQFQu7w...,MBA,2000 - 2002,https://www.linkedin.com/in/aamer-naseer-9065899/,NaN
1,1,NaN,FG School Okara Cantt,NaN,NaN,NaN,https://www.linkedin.com/in/aamer-naseer-9065899/,NaN
2,2,https://www.linkedin.com/company/667986/,Institute of Business Administration,https://media.licdn.com/dms/image/C4D0BAQGP1gR...,NaN,NaN,https://www.linkedin.com/in/aamer-naseer-9065899/,NaN
3,3,https://www.linkedin.com/company/4477/,University of Oxford,https://media.licdn.com/dms/image/C560BAQGwPZs...,"DPhil, Inorganic Chemistry",2001 - 2004,https://www.linkedin.com/in/aamir-khan-48a1541/,NaN
4,4,https://www.linkedin.com/company/4477/,University of Oxford,https://media.licdn.com/dms/image/C560BAQGwPZs...,"MChem, Chemistry",1997 - 2001,https://www.linkedin.com/in/aamir-khan-48a1541/,NaN


In [56]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=x)

https://docs.google.com/spreadsheets/d/1jkjm8YRguLoSvLY-GWS0cCq-Wo8R8iEksMDF9KvwekA/edit#gid=0


In [57]:
sheet2 = sheets.InteractiveSheet(df=y)

https://docs.google.com/spreadsheets/d/13sntEQ-yXwjWI4ItYAC15D_WWJJmuEIxYpwZG1nnwn0/edit#gid=0


In [54]:
y.head()

,Unnamed: 0,schoolUrl,schoolName,logoUrl,degree,dateRange,url,description
0,0,https://www.linkedin.com/company/6926/,Queen's University,https://media.licdn.com/dms/image/C4E0BAQE7P2b...,Master of Business Administration (MBA),2016 - 2017,https://www.linkedin.com/in/jasonbanducci/,NaN
1,1,https://www.linkedin.com/company/166673/,Universität St. Gallen-Hochschule für Wirtscha...,https://media.licdn.com/dms/image/C560BAQFwnhW...,MBA Exchange - International Study Program (ISP),2016 - 2016,https://www.linkedin.com/in/jasonbanducci/,NaN
2,2,https://www.linkedin.com/company/166689/,Western University,https://media.licdn.com/dms/image/C560BAQExbN4...,Bachelor of Management and Organizational Stud...,2008 - 2012,https://www.linkedin.com/in/jasonbanducci/,NaN
3,3,https://www.linkedin.com/company/2584/,University of Washington,https://media.licdn.com/dms/image/C4D0BAQEMmhF...,"Bachelor of Business Administration (B.B.A.), ...",1998 - 2002,https://www.linkedin.com/in/jason-black-8880904b/,NaN
4,4,https://www.linkedin.com/company/3523/,Cornell University,https://media.licdn.com/dms/image/C560BAQE8SOX...,"Bachelor's degree, Applied Economics and Busin...",1996,https://www.linkedin.com/in/jason-braunstein-0...,NaN
